# Audio File Visualization Notebook

This notebook loads paired audio files from the SignalTrain LA2A dataset and visualizes them with interactive plots.

**File pairing pattern:**
- Input: `input_<num>_.wav`
- Target: `target_<num>_LA2A_<s1>__<s2>__<s3>.wav`

The notebook loads audio sections on demand to handle large files (~200MB each).

In [1]:
import os
import re
from pathlib import Path
import numpy as np
import soundfile as sf
import matplotlib.pyplot as plt
from matplotlib.widgets import Slider
import warnings
warnings.filterwarnings('ignore')

# Enable interactive matplotlib backend for widgets
# Try widget backend first (requires: pip install ipympl)
try:
    from IPython import get_ipython
    ipython = get_ipython()
    if ipython is not None:
        try:
            ipython.run_line_magic('matplotlib', 'widget')
            print("Using matplotlib widget backend")
        except:
            try:
                ipython.run_line_magic('matplotlib', 'notebook')
                print("Using matplotlib notebook backend")
            except:
                ipython.run_line_magic('matplotlib', 'inline')
                print("Warning: Using inline backend. Install ipympl for better interactivity: pip install ipympl")
    else:
        plt.ion()  # Interactive mode for non-IPython environments
        print("Using matplotlib interactive mode")
except Exception as e:
    plt.ion()
    print(f"Using matplotlib interactive mode (fallback): {e}")

# Configuration
DATASET_DIR = "/home/shreyan/Documents/DATASETS/SignalTrain_LA2A_Dataset_1.1/Train"

Using matplotlib notebook backend


In [2]:
def find_paired_files(dataset_dir):
    """
    Find all paired input and target audio files.
    
    Returns:
        List of tuples: [(num, input_path, target_path, states), ...]
    """
    dataset_path = Path(dataset_dir)
    if not dataset_path.exists():
        raise FileNotFoundError(f"Dataset directory not found: {dataset_dir}")
    
    # Pattern for input files: input_<num>_.wav
    input_pattern = re.compile(r'input_(\d+)_\.wav')
    # Pattern for target files: target_<num>_LA2A_<s1>__<s2>__<s3>.wav
    target_pattern = re.compile(r'target_(\d+)_LA2A_([^_]+)__([^_]+)__([^_]+)\.wav')
    
    input_files = {}
    target_files = {}
    
    # Find all input files
    for file_path in dataset_path.glob("input_*.wav"):
        match = input_pattern.match(file_path.name)
        if match:
            num = int(match.group(1))
            input_files[num] = file_path
    
    # Find all target files
    for file_path in dataset_path.glob("target_*_LA2A_*.wav"):
        match = target_pattern.match(file_path.name)
        if match:
            num = int(match.group(1))
            s1, s2, s3 = match.groups()[1:]
            target_files[num] = (file_path, (s1, s2, s3))
    
    # Find pairs
    pairs = []
    for num in sorted(set(input_files.keys()) & set(target_files.keys())):
        input_path = input_files[num]
        target_path, states = target_files[num]
        pairs.append((num, input_path, target_path, states))
    
    return pairs

# Find all paired files
pairs = find_paired_files(DATASET_DIR)
print(f"Found {len(pairs)} paired audio files")
if len(pairs) > 0:
    print(f"\nFirst few pairs:")
    for num, input_path, target_path, states in pairs[:5]:
        print(f"  Pair {num}: {input_path.name} <-> {target_path.name} (states: {states})")

Found 66 paired audio files

First few pairs:
  Pair 138: input_138_.wav <-> target_138_LA2A_3c__0__0.wav (states: ('3c', '0', '0'))
  Pair 139: input_139_.wav <-> target_139_LA2A_3c__0__5.wav (states: ('3c', '0', '5'))
  Pair 141: input_141_.wav <-> target_141_LA2A_3c__0__15.wav (states: ('3c', '0', '15'))
  Pair 142: input_142_.wav <-> target_142_LA2A_3c__0__20.wav (states: ('3c', '0', '20'))
  Pair 143: input_143_.wav <-> target_143_LA2A_3c__0__25.wav (states: ('3c', '0', '25'))


In [3]:
def get_audio_info(file_path):
    """Get audio file information without loading the entire file."""
    with sf.SoundFile(str(file_path)) as f:
        return {
            'sample_rate': f.samplerate,
            'channels': f.channels,
            'frames': len(f),
            'duration': len(f) / f.samplerate,
            'subtype': f.subtype
        }

# Get info for first pair as example
if len(pairs) > 0:
    num, input_path, target_path, states = pairs[0]
    print(f"Pair {num} information:")
    print(f"\nInput file: {input_path.name}")
    input_info = get_audio_info(input_path)
    for key, value in input_info.items():
        print(f"  {key}: {value}")
    
    print(f"\nTarget file: {target_path.name}")
    target_info = get_audio_info(target_path)
    for key, value in target_info.items():
        print(f"  {key}: {value}")

Pair 138 information:

Input file: input_138_.wav
  sample_rate: 44100
  channels: 1
  frames: 52920000
  duration: 1200.0
  subtype: FLOAT

Target file: target_138_LA2A_3c__0__0.wav
  sample_rate: 44100
  channels: 1
  frames: 52920000
  duration: 1200.0
  subtype: FLOAT


In [4]:
def load_audio_section(file_path, start_sample=0, num_samples=None, sample_rate=None):
    """
    Load a specific section of an audio file without loading the entire file.
    
    Args:
        file_path: Path to audio file
        start_sample: Starting sample index (default: 0)
        num_samples: Number of samples to load (default: None = load to end)
        sample_rate: If provided, convert start_sample and num_samples from seconds
    
    Returns:
        audio_data: numpy array of audio samples
        actual_sample_rate: sample rate of the file
    """
    with sf.SoundFile(str(file_path)) as f:
        actual_sample_rate = f.samplerate
        
        # Convert from seconds to samples if sample_rate is provided
        if sample_rate is not None:
            start_sample = int(start_sample * actual_sample_rate)
            if num_samples is not None:
                num_samples = int(num_samples * actual_sample_rate)
        
        # Ensure start_sample is within bounds
        start_sample = max(0, min(start_sample, len(f)))
        
        # If num_samples is None, load to the end
        if num_samples is None:
            num_samples = len(f) - start_sample
        else:
            # Ensure we don't read past the end
            num_samples = min(num_samples, len(f) - start_sample)
        
        # Seek to start position and read
        f.seek(start_sample)
        audio_data = f.read(num_samples)
        
        # If stereo, take first channel
        if len(audio_data.shape) > 1:
            audio_data = audio_data[:, 0]
        
        return audio_data, actual_sample_rate

# Test loading a small section
if len(pairs) > 0:
    num, input_path, target_path, states = pairs[0]
    print(f"Loading first 1 second of pair {num}...")
    
    input_audio, sr = load_audio_section(input_path, start_sample=0, num_samples=1.0, sample_rate=1.0)
    target_audio, _ = load_audio_section(target_path, start_sample=0, num_samples=1.0, sample_rate=1.0)
    
    print(f"Loaded {len(input_audio)} samples from input ({len(input_audio)/sr:.2f} seconds)")
    print(f"Loaded {len(target_audio)} samples from target ({len(target_audio)/sr:.2f} seconds)")
    print(f"Sample rate: {sr} Hz")

Loading first 1 second of pair 138...
Loaded 44100 samples from input (1.00 seconds)
Loaded 44100 samples from target (1.00 seconds)
Sample rate: 44100 Hz


In [5]:
class AudioVisualizer:
    """Interactive audio visualizer for paired audio files."""
    
    def __init__(self, input_path, target_path, sample_rate=None, window_duration=5.0):
        """
        Initialize the visualizer.
        
        Args:
            input_path: Path to input audio file
            target_path: Path to target audio file
            sample_rate: Sample rate (will be detected if None)
            window_duration: Duration of audio to display in seconds (default: 5.0)
        """
        self.input_path = Path(input_path)
        self.target_path = Path(target_path)
        self.window_duration = window_duration
        
        # Get audio info
        input_info = get_audio_info(self.input_path)
        target_info = get_audio_info(self.target_path)
        
        self.sample_rate = sample_rate or input_info['sample_rate']
        self.input_duration = input_info['duration']
        self.target_duration = target_info['duration']
        self.max_duration = max(self.input_duration, self.target_duration)
        
        # Current view parameters
        self.current_start_time = 0.0
        self.current_window_duration = window_duration
        
        # Load initial data
        self.input_audio = None
        self.target_audio = None
        self._load_current_section()
        
        # Setup plot
        self.fig, (self.ax1, self.ax2) = plt.subplots(2, 1, figsize=(14, 8), sharex=True)
        self._setup_plot()
        
    def _load_current_section(self):
        """Load the current section of audio."""
        start_sample = self.current_start_time
        num_samples = self.current_window_duration
        
        self.input_audio, _ = load_audio_section(
            self.input_path, 
            start_sample=start_sample, 
            num_samples=num_samples, 
            sample_rate=self.sample_rate
        )
        self.target_audio, _ = load_audio_section(
            self.target_path, 
            start_sample=start_sample, 
            num_samples=num_samples, 
            sample_rate=self.sample_rate
        )
        
    def _setup_plot(self):
        """Setup the matplotlib plot with sliders."""
        self.ax1.clear()
        self.ax2.clear()
        
        # Create time axis
        time_axis = np.arange(len(self.input_audio)) / self.sample_rate + self.current_start_time
        
        # Plot waveforms
        self.line1, = self.ax1.plot(time_axis, self.input_audio, 'b-', linewidth=0.5, alpha=0.7)
        self.line2, = self.ax2.plot(time_axis, self.target_audio, 'r-', linewidth=0.5, alpha=0.7)
        
        # Labels and titles
        self.ax1.set_ylabel('Amplitude', fontsize=12)
        self.ax1.set_title(f'Input Audio: {self.input_path.name}', fontsize=14, fontweight='bold')
        self.ax1.grid(True, alpha=0.3)
        
        self.ax2.set_xlabel('Time (seconds)', fontsize=12)
        self.ax2.set_ylabel('Amplitude', fontsize=12)
        self.ax2.set_title(f'Target Audio: {self.target_path.name}', fontsize=14, fontweight='bold')
        self.ax2.grid(True, alpha=0.3)
        
        # Adjust layout to make room for sliders
        plt.subplots_adjust(bottom=0.25)
        
        # Calculate slider limits
        max_start = max(0.0, self.max_duration - self.current_window_duration)
        max_duration = min(30.0, self.max_duration)
        
        # Create sliders
        ax_slider_time = plt.axes([0.15, 0.1, 0.7, 0.03])
        ax_slider_duration = plt.axes([0.15, 0.05, 0.7, 0.03])
        
        self.slider_time = Slider(
            ax_slider_time, 
            'Start Time (s)', 
            0.0, 
            max_start if max_start > 0 else 1.0,
            valinit=self.current_start_time,
            valstep=0.1
        )
        
        self.slider_duration = Slider(
            ax_slider_duration,
            'Window Duration (s)',
            0.1,
            max_duration,
            valinit=self.current_window_duration,
            valstep=0.1
        )
        
        # Connect sliders to update function
        self.slider_time.on_changed(self._update_time)
        self.slider_duration.on_changed(self._update_duration)
        
        plt.tight_layout()
        plt.show(block=False)  # Don't block, allow interaction
        
    def _update_time(self, val):
        """Update plot when time slider changes."""
        self.current_start_time = val
        # Ensure we don't go past the end
        max_start = max(0.0, self.max_duration - self.current_window_duration)
        self.current_start_time = min(self.current_start_time, max_start)
        if self.current_start_time != val:
            self.slider_time.set_val(self.current_start_time)
        self._update_plot()
        
    def _update_duration(self, val):
        """Update plot when duration slider changes."""
        self.current_window_duration = val
        # Ensure we don't go past the end
        max_start = max(0.0, self.max_duration - self.current_window_duration)
        if self.current_start_time > max_start:
            self.current_start_time = max_start
            self.slider_time.set_val(self.current_start_time)
        self.slider_time.valmax = max_start
        self._update_plot()
        
    def _update_plot(self):
        """Reload audio and update the plot."""
        self._load_current_section()
        
        # Update time axis
        time_axis = np.arange(len(self.input_audio)) / self.sample_rate + self.current_start_time
        
        # Update plot data
        self.line1.set_data(time_axis, self.input_audio)
        self.line2.set_data(time_axis, self.target_audio)
        
        # Update axis limits
        self.ax1.set_xlim(time_axis[0], time_axis[-1])
        self.ax2.set_xlim(time_axis[0], time_axis[-1])
        
        # Update y-axis limits
        self.ax1.set_ylim(self.input_audio.min() * 1.1, self.input_audio.max() * 1.1)
        self.ax2.set_ylim(self.target_audio.min() * 1.1, self.target_audio.max() * 1.1)
        
        self.fig.canvas.draw_idle()
        self.fig.canvas.flush_events()  # Force update
        
    def show(self):
        """Display the interactive plot."""
        plt.show(block=False)

print("AudioVisualizer class defined. Ready to visualize audio files!")

AudioVisualizer class defined. Ready to visualize audio files!


## Interactive Visualizer with ipywidgets

This version uses ipywidgets which works better in JupyterLab and Jupyter Notebook.

**Note:** If widgets don't appear, you may need to install the JupyterLab extension:
```bash
pip install jupyterlab-widgets
# Then restart JupyterLab
```

In [9]:
try:
    import ipywidgets as widgets
    from IPython.display import display
    IPYWIDGETS_AVAILABLE = True
    print("✓ ipywidgets is available")
except ImportError:
    IPYWIDGETS_AVAILABLE = False
    print("✗ ipywidgets not available. Install with: pip install ipywidgets")

# Ensure matplotlib uses inline backend for ipywidgets
if IPYWIDGETS_AVAILABLE:
    try:
        from IPython import get_ipython
        ipython = get_ipython()
        if ipython is not None:
            ipython.run_line_magic('matplotlib', 'inline')
            print("✓ Matplotlib backend set to inline")
    except Exception as e:
        print(f"Note: Could not set matplotlib backend: {e}")

# Check for JupyterLab widgets extension
try:
    import jupyterlab_widgets
    JUPYTERLAB_WIDGETS_AVAILABLE = True
    print("✓ jupyterlab-widgets extension is available")
except ImportError:
    JUPYTERLAB_WIDGETS_AVAILABLE = False
    print("⚠ jupyterlab-widgets not found. Install with: uv pip install jupyterlab-widgets")
    print("  (Then restart JupyterLab for widgets to work)")

# Quick test to verify widgets work
if IPYWIDGETS_AVAILABLE:
    test_slider = widgets.IntSlider(value=5, min=0, max=10, description='Test:')
    print("\nTesting widget display (you should see a slider below):")
    display(test_slider)
    print("If you see the slider above, widgets are working!")
    if not JUPYTERLAB_WIDGETS_AVAILABLE:
        print("\n⚠ NOTE: If you don't see the slider, install jupyterlab-widgets and restart JupyterLab")

def create_interactive_plot(input_path, target_path, window_duration=5.0):
    """
    Create an interactive plot using ipywidgets (works better in Jupyter).
    
    Args:
        input_path: Path to input audio file
        target_path: Path to target audio file
        window_duration: Initial window duration in seconds
    """
    if not IPYWIDGETS_AVAILABLE:
        print("ipywidgets not available. Using matplotlib widgets instead.")
        visualizer = AudioVisualizer(input_path, target_path, window_duration=window_duration)
        visualizer.show()
        return visualizer
    
    input_path = Path(input_path)
    target_path = Path(target_path)
    
    # Get audio info
    input_info = get_audio_info(input_path)
    target_info = get_audio_info(target_path)
    
    sample_rate = input_info['sample_rate']
    max_duration = max(input_info['duration'], target_info['duration'])
    
    # Create widgets
    time_slider = widgets.FloatSlider(
        value=0.0,
        min=0.0,
        max=max(0.0, max_duration - window_duration),
        step=0.1,
        description='Start Time (s):',
        style={'description_width': 'initial'},
        layout=widgets.Layout(width='80%')
    )
    
    duration_slider = widgets.FloatSlider(
        value=window_duration,
        min=0.1,
        max=min(30.0, max_duration),
        step=0.1,
        description='Duration (s):',
        style={'description_width': 'initial'},
        layout=widgets.Layout(width='80%')
    )
    
    # Create output widget for plot
    output = widgets.Output()
    
    def update_plot(start_time, duration):
        """Update the plot based on slider values."""
        with output:
            output.clear_output(wait=True)
            
            # Load audio sections
            input_audio, sr = load_audio_section(
                input_path, 
                start_sample=start_time, 
                num_samples=duration, 
                sample_rate=sample_rate
            )
            target_audio, _ = load_audio_section(
                target_path, 
                start_sample=start_time, 
                num_samples=duration, 
                sample_rate=sample_rate
            )
            
            # Create time axis
            time_axis = np.arange(len(input_audio)) / sr + start_time
            
            # Create plot
            fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 8), sharex=True)
            
            ax1.plot(time_axis, input_audio, 'b-', linewidth=0.5, alpha=0.7)
            ax1.set_ylim(-1, 1)
            ax1.set_ylabel('Amplitude', fontsize=12)
            ax1.set_title(f'Input Audio: {input_path.name}', fontsize=14, fontweight='bold')
            ax1.grid(True, alpha=0.3)
            
            ax2.plot(time_axis, target_audio, 'r-', linewidth=0.5, alpha=0.7)
            ax2.set_ylim(-1, 1)
            ax2.set_xlabel('Time (seconds)', fontsize=12)
            ax2.set_ylabel('Amplitude', fontsize=12)
            ax2.set_title(f'Target Audio: {target_path.name}', fontsize=14, fontweight='bold')
            ax2.grid(True, alpha=0.3)
            
            plt.tight_layout()
            plt.show()  # This will render in the output widget with inline backend
    
    # Connect widgets
    def on_time_change(change):
        # Update duration slider max based on current time
        max_start = max(0.0, max_duration - duration_slider.value)
        time_slider.max = max_start
        if time_slider.value > max_start:
            time_slider.value = max_start
        update_plot(time_slider.value, duration_slider.value)
    
    def on_duration_change(change):
        # Update time slider max based on current duration
        max_start = max(0.0, max_duration - duration_slider.value)
        time_slider.max = max_start
        if time_slider.value > max_start:
            time_slider.value = max_start
        update_plot(time_slider.value, duration_slider.value)
    
    time_slider.observe(on_time_change, names='value')
    duration_slider.observe(on_duration_change, names='value')
    
    # Create the widget container
    widget_container = widgets.VBox([
        widgets.HTML(f"<h3>Interactive Audio Visualization</h3>"),
        widgets.HTML(f"<p>Input: {input_path.name}<br>Target: {target_path.name}</p>"),
        time_slider,
        duration_slider,
        output
    ])
    
    # Display widgets first
    display(widget_container)
    
    # Then create initial plot (this will appear in the output widget)
    update_plot(time_slider.value, duration_slider.value)
    
    return time_slider, duration_slider, output, widget_container

print("Interactive plot function with ipywidgets ready!")

✓ ipywidgets is available
✓ Matplotlib backend set to inline
✓ jupyterlab-widgets extension is available

Testing widget display (you should see a slider below):


IntSlider(value=5, description='Test:', max=10)

If you see the slider above, widgets are working!
Interactive plot function with ipywidgets ready!


## Visualize Audio Files

Select a pair number to visualize. The visualizer will load audio sections on demand.

**For interactive plots (with sliders):**
1. Install in your kernel environment: `uv pip install jupyterlab-widgets` (or `pip install jupyterlab-widgets`)
2. **Restart JupyterLab** (important!)
3. Re-run the cells below

**For static plots (no sliders) - WORKS NOW:**
- Just change `START_TIME` and `DURATION` variables below and re-run the cell
- This works without any additional packages and will always display

In [12]:
# Select which pair to visualize (change this number)
PAIR_INDEX = 60  # Index in the pairs list (0-based)

# You can also customize these parameters:
START_TIME = 0.0  # Start time in seconds
DURATION = 5.0    # Duration to display in seconds

if PAIR_INDEX < len(pairs):
    num, input_path, target_path, states = pairs[PAIR_INDEX]
    print(f"Visualizing pair {num}")
    print(f"Input: {input_path.name}")
    print(f"Target: {target_path.name}")
    print(f"States: {states}")
    
    # Try ipywidgets version first (works better in Jupyter)
    if IPYWIDGETS_AVAILABLE:
        # Check if jupyterlab-widgets is available
        try:
            import jupyterlab_widgets
            use_interactive = True
        except ImportError:
            use_interactive = False
            print("\n" + "="*60)
            print("⚠ jupyterlab-widgets not installed in this kernel.")
            print("For interactive plots, run in a terminal:")
            print("  uv pip install jupyterlab-widgets")
            print("Then restart JupyterLab and re-run this cell.")
            print("="*60)
            print("\nUsing static plot instead (change START_TIME/DURATION to view different sections):\n")
        
        if use_interactive:
            print("\nCreating interactive visualizer with ipywidgets...")
            print("(You should see sliders and a plot below)")
            try:
                create_interactive_plot(input_path, target_path, window_duration=DURATION)
            except Exception as e:
                print(f"Error creating interactive plot: {e}")
                print("Falling back to static plot...")
                plot_audio_section(input_path, target_path, start_time=START_TIME, duration=DURATION)
        else:
            # Use static plot
            plot_audio_section(input_path, target_path, start_time=START_TIME, duration=DURATION)
    else:
        print("\n" + "="*60)
        print("NOTE: ipywidgets is not installed.")
        print("For interactive plots, install: pip install ipywidgets")
        print("For JupyterLab, also install: pip install jupyterlab-widgets")
        print("="*60)
        print("\nShowing static plot instead. Change START_TIME and DURATION above to view different sections.")
        print("Or re-run this cell with different values.\n")
        # Use static plot function
        plot_audio_section(input_path, target_path, start_time=START_TIME, duration=DURATION)
else:
    print(f"Pair index {PAIR_INDEX} out of range. Available pairs: 0-{len(pairs)-1}")

Visualizing pair 254
Input: input_254_.wav
Target: target_254_LA2A_2c__1__55.wav
States: ('2c', '1', '55')

Creating interactive visualizer with ipywidgets...
(You should see sliders and a plot below)


## Alternative: Static Plot with Custom Section

If you prefer a static plot without sliders, you can plot a specific section directly:

In [8]:
def plot_audio_section(input_path, target_path, start_time=0.0, duration=5.0, sample_rate=None):
    """
    Plot a specific section of paired audio files.
    
    Args:
        input_path: Path to input audio file
        target_path: Path to target audio file
        start_time: Start time in seconds (default: 0.0)
        duration: Duration to plot in seconds (default: 5.0)
        sample_rate: Sample rate (will be detected if None)
    """
    # Load audio sections
    input_audio, sr = load_audio_section(input_path, start_sample=start_time, num_samples=duration, sample_rate=sample_rate)
    target_audio, _ = load_audio_section(target_path, start_sample=start_time, num_samples=duration, sample_rate=sample_rate)
    
    # Create time axis
    time_axis = np.arange(len(input_audio)) / sr + start_time
    
    # Create subplots
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 8), sharex=True)
    
    # Plot waveforms
    ax1.plot(time_axis, input_audio, 'b-', linewidth=0.5, alpha=0.7)
    ax1.set_ylabel('Amplitude', fontsize=12)
    ax1.set_title(f'Input Audio: {Path(input_path).name}', fontsize=14, fontweight='bold')
    ax1.grid(True, alpha=0.3)
    
    ax2.plot(time_axis, target_audio, 'r-', linewidth=0.5, alpha=0.7)
    ax2.set_xlabel('Time (seconds)', fontsize=12)
    ax2.set_ylabel('Amplitude', fontsize=12)
    ax2.set_title(f'Target Audio: {Path(target_path).name}', fontsize=14, fontweight='bold')
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    return input_audio, target_audio, sr

# Example: Plot first 5 seconds of the first pair
if len(pairs) > 0:
    num, input_path, target_path, states = pairs[0]
    print(f"Plotting first 5 seconds of pair {num}...")
    plot_audio_section(input_path, target_path, start_time=0.0, duration=5.0)

Plotting first 5 seconds of pair 138...


TypeError: 'float' object cannot be interpreted as an integer

## List All Available Pairs

Use this cell to see all available pairs and their file names:

In [ ]:
# Display all available pairs
print(f"Total pairs found: {len(pairs)}\n")
print("Available pairs:")
print("-" * 100)
for idx, (num, input_path, target_path, states) in enumerate(pairs):
    input_info = get_audio_info(input_path)
    target_info = get_audio_info(target_path)
    print(f"Index {idx:3d} | Pair {num:3d} | States: {states}")
    print(f"         Input:  {input_path.name} ({input_info['duration']:.2f}s, {input_info['sample_rate']}Hz)")
    print(f"         Target: {target_path.name} ({target_info['duration']:.2f}s, {target_info['sample_rate']}Hz)")
    print()

Total pairs found: 66

Available pairs:
----------------------------------------------------------------------------------------------------
Index   0 | Pair 138 | States: ('3c', '0', '0')
         Input:  input_138_.wav (1200.00s, 44100Hz)
         Target: target_138_LA2A_3c__0__0.wav (1200.00s, 44100Hz)

Index   1 | Pair 139 | States: ('3c', '0', '5')
         Input:  input_139_.wav (1200.00s, 44100Hz)
         Target: target_139_LA2A_3c__0__5.wav (1200.00s, 44100Hz)

Index   2 | Pair 141 | States: ('3c', '0', '15')
         Input:  input_141_.wav (1200.00s, 44100Hz)
         Target: target_141_LA2A_3c__0__15.wav (1200.00s, 44100Hz)

Index   3 | Pair 142 | States: ('3c', '0', '20')
         Input:  input_142_.wav (1200.00s, 44100Hz)
         Target: target_142_LA2A_3c__0__20.wav (1200.00s, 44100Hz)

Index   4 | Pair 143 | States: ('3c', '0', '25')
         Input:  input_143_.wav (1200.00s, 44100Hz)
         Target: target_143_LA2A_3c__0__25.wav (1200.00s, 44100Hz)

Index   5 | Pair 1